# Linear Regression
- Non-linear data
- linear model
- y = 2x + 3

### 1. Import

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable

In [2]:
from visdom import Visdom
viz = Visdom() 

Setting up a new session...


### 2. Generate data

In [36]:
num_data = 1000
num_epoch = 1000
noise = init.normal_(torch.FloatTensor(num_data, 1), std=10)
x = init.uniform_(torch.Tensor(num_data, 1), -10, 6)
y = -x**3 - 8*(x**2) + 3
y_noise = y + noise

In [37]:
input_data = torch.cat([x, y_noise], 1)
input_data.size() 

torch.Size([1000, 2])

#### Visualize data with visdom

In [38]:
win = viz.scatter(
    X = input_data, 
    opts=dict(
        xtickmin=-10,
        xtickmax=6,
        xtickstep=1,
        ytickmin=-600,
        ytickmax=300,
        ytickstep=1,
        markersymbol='dot',
        markersize=5,
        markercolor=np.random.randint(0, 255, num_data)
    )
)

In [39]:
origin_data = torch.cat([x, y], 1)
win = viz.scatter(X = origin_data, win=win, update='append')

### 3. Model & Optimizer

In [40]:
model = nn.Linear(1,1)
output = model(Variable(x))

loss_func = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01) 

### 4. Train

In [41]:
loss_arr = []
label = Variable(y_noise)

for i in range(num_epoch):
    optimizer.zero_grad()
    output = model(Variable(x))
    
    loss = loss_func(output, label) 
    loss.backward()
    optimizer.step()
    
    if i % 10 == 0:
        print('epoch No', i, loss)
        
    loss_arr.append(loss.data.numpy())

epoch No 0 tensor(20359.4688, grad_fn=<MseLossBackward>)
epoch No 10 tensor(13857.6006, grad_fn=<MseLossBackward>)
epoch No 20 tensor(12064.9473, grad_fn=<MseLossBackward>)
epoch No 30 tensor(10808.5889, grad_fn=<MseLossBackward>)
epoch No 40 tensor(9928.0850, grad_fn=<MseLossBackward>)
epoch No 50 tensor(9310.9922, grad_fn=<MseLossBackward>)
epoch No 60 tensor(8878.5088, grad_fn=<MseLossBackward>)
epoch No 70 tensor(8575.4072, grad_fn=<MseLossBackward>)
epoch No 80 tensor(8362.9824, grad_fn=<MseLossBackward>)
epoch No 90 tensor(8214.1055, grad_fn=<MseLossBackward>)
epoch No 100 tensor(8109.7686, grad_fn=<MseLossBackward>)
epoch No 110 tensor(8036.6440, grad_fn=<MseLossBackward>)
epoch No 120 tensor(7985.3970, grad_fn=<MseLossBackward>)
epoch No 130 tensor(7949.4800, grad_fn=<MseLossBackward>)
epoch No 140 tensor(7924.3071, grad_fn=<MseLossBackward>)
epoch No 150 tensor(7906.6655, grad_fn=<MseLossBackward>)
epoch No 160 tensor(7894.3018, grad_fn=<MseLossBackward>)
epoch No 170 tensor(7

### 5. Check trained parameters

In [42]:
param_list = list(model.parameters())
print(param_list[0].data, param_list[1].data) 

tensor([[-18.7751]]) tensor([-98.1028])


### 6. Visualize output

In [43]:
win_2 = viz.scatter(
    X = input_data,
    opts=dict(
        xtickmin=-10,
        xtickmax=6,
        xtickstep=1,
        ytickmin=-600,
        ytickmax=300,
        ytickstep=1,
        markersymbol='dot',
        markercolor=np.random.randint(0, 255, num_data),
        markersize=5,
    ),
)

In [44]:
output_data = torch.cat([x, output.data], 1)
win = viz.scatter(X = output_data, win=win_2, update='append')

### 7. Visualize loss graph

In [45]:
x = np.reshape([i for i in range(num_epoch)], newshape=[num_epoch])
loss_data = np.reshape(loss_arr, newshape=[num_epoch])

win_3 = viz.line(X = x, Y = loss_data)